<a href="https://colab.research.google.com/github/BimiLevi/Covid19/blob/master/Riskified_task_Jul_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Riskified Home Test - Tal Levi

In [ ]:
%config IPCompleter.greedy=True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## EDA

In [ ]:
# Choose the right path for uploading the CSV file
# df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MH_home_task_dataset.csv')
df = pd.read_csv('/content/MH_home_task_dataset.csv')

In [ ]:
df.shape

(40825, 10)

In [ ]:
df.columns

Index(['order_id', 'order_date', 'order_status', 'price', 'digital_product',
       'customer_account_age', 'order_source', 'billing_zip',
       'shipping_name_length', 'classification_score'],
      dtype='object')

In [ ]:
df.order_date = pd.to_datetime(df.order_date, format = '%Y-%m-%d')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40825 entries, 0 to 40824
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   order_id              40825 non-null  int64         
 1   order_date            40825 non-null  datetime64[ns]
 2   order_status          40825 non-null  object        
 3   price                 40825 non-null  float64       
 4   digital_product       40825 non-null  bool          
 5   customer_account_age  40825 non-null  int64         
 6   order_source          40825 non-null  object        
 7   billing_zip           40825 non-null  object        
 8   shipping_name_length  40825 non-null  int64         
 9   classification_score  40825 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 2.8+ MB


In [ ]:
df.head(5)

,order_id,order_date,order_status,price,digital_product,customer_account_age,order_source,billing_zip,shipping_name_length,classification_score
0,906967102,2019-06-01,approved,213.88,False,0,web,33062,13,0.999219
1,906987974,2019-06-01,approved,50.00,True,0,mobile_app,48317,0,0.995886
2,906994260,2019-06-01,approved,100.00,True,328,web,92124,0,0.966901
3,906997653,2019-06-01,approved,25.00,True,1371,web,95448,0,0.999854
4,907408483,2019-06-01,approved,2035.76,False,139,web,61107,17,0.999965


In [ ]:
df.tail(5)

,order_id,order_date,order_status,price,digital_product,customer_account_age,order_source,billing_zip,shipping_name_length,classification_score
40820,950171894,2019-07-01,approved,107.44,False,84,web,37620,8,0.999743
40821,950197404,2019-07-01,approved,92.85,True,5159,web,33134,0,0.999964
40822,950197813,2019-07-01,approved,1249.86,False,196,web,21901,11,0.999912
40823,950221337,2019-07-01,approved,310.44,False,938,web,94109,9,0.997374
40824,950442061,2019-07-01,approved,211.90,True,0,web,94541,0,0.986633


In [ ]:
df.describe()

,order_id,price,customer_account_age,shipping_name_length,classification_score
count,4.082500e+04,40825.000000,40825.000000,40825.000000,40825.000000
mean,9.288490e+08,396.114069,736.804556,4.036375,0.943287
std,1.152215e+07,771.641853,1171.305366,5.901319,0.168241
min,9.068728e+08,0.000000,-3.000000,0.000000,0.038408
25%,9.200580e+08,35.000000,0.000000,0.000000,0.988642
50%,9.291292e+08,82.440000,214.000000,0.000000,0.999177
75%,9.377158e+08,280.210000,980.000000,10.000000,0.999895
max,9.505111e+08,11033.310000,6298.000000,21.000000,1.000000


## Task 1

 Rely exclusively on model scores using Riskified’s decision-engine, and set a decline
threshold that will provide a 90% approval rate

In [ ]:
df = df.sort_values(by='classification_score', ascending = False) # sorting the DF in DESC order

In [ ]:
top_90_index = int(np.ceil(0.9 * df.shape[0])) # finding the index of the X90% value
top_90_index

36743

In [ ]:
threshold_index = df.iloc[top_90_index] # getting first the raw in the qunitl
threshold_index

order_id                          942489066
order_date              2019-06-26 00:00:00
order_status                       approved
price                                 100.0
digital_product                        True
customer_account_age                   1439
order_source                            web
billing_zip                           67601
shipping_name_length                      0
classification_score               0.860724
Name: 32611, dtype: object

In [ ]:
print(f'threshold score is {threshold_index.classification_score}') # Getting the threshold score

threshold score is 0.860723546814588


## Task 2

Plot the model scores distribution

In [ ]:
fig = px.histogram(df, x="classification_score",
                   title = 'Scores Distribution',
                   labels = {'classification_score' : 'Classification Score',
                            'order_status' : 'Order Status'},
                   color = 'order_status'
                  )
fig.show()

When it comes to credit card fraud, the data is usually skewed , that's because most transactions are legit.

## Task 3

Assuming we aim at a proportion of 50% between the CHB cost and the total revenue
(sum amount of CHBs divided by the total revenue), what would have to be the fee based
Riskified’s business model?

steps:
1. find the CHB cost (sum of prices for all CHB orders).
2. calculate the revenue according to the desired proportion -> total revenue = CHB sum / 0.5
3. based on Riskfied model the fee is a "small fraction (fee) of the amount of all approved orders", therefore by adding the total revenue with the sum of CHB cost, we will get the full amount of the original fee.

In [ ]:
chb_cost = df[df.order_status != 'approved'].price.sum()
chb_cost

48637.119999999995

In [ ]:
total_rev = chb_cost/0.5 # revnue calculation
total_rev

97274.23999999999

In [ ]:
fee = total_rev + chb_cost
fee

145911.36

In [ ]:
100 * fee / df.price.sum() # fee percent

0.9022827296028657

## Task 4

In the dataset, there’s a column indicating whether the purchased goods are digital or
tangible. What can you say about the risk level of each segment?

In [ ]:
norm_v_count = df.digital_product.value_counts(normalize=True) * 100
v_count = df.digital_product.value_counts()
pd.concat([v_count,norm_v_count], axis=1, keys=['counts', '%'])

,counts,%
True,27287,66.838947
False,13538,33.161053


In [ ]:
# checking the distribution between each group and subgroup
100 * df.groupby(by=['digital_product','order_status']).digital_product.count() /df.groupby(by=['digital_product']).digital_product.count()

digital_product  order_status
False            approved        99.704535
                 chargeback       0.295465
True             approved        98.827280
                 chargeback       1.172720
Name: digital_product, dtype: float64

Two-thirds of all orders are from digital products, when comparing the ratio of CHB per product type the data shows that digital products have a higher rate of CHB (1.172% >> 0.295%). Therefore it looks like digital products are more risky then non-digital products.

## Task 5

Free exploration - Provide 2-3 interesting breakdowns or insights

In [ ]:
# 1. CHB per billing zip - highest zip codes with CHB
chb = df.loc[df['order_status'] != 'approved']
filtered_chb = chb.groupby(by=['order_status', 'billing_zip']).filter(lambda x: len(x) >= 2).groupby(by=['order_status', 'billing_zip']).size()
filtered_chb = filtered_chb.reset_index().rename(columns = {0:'chb_count'}).sort_values(by='chb_count',ascending = False).head(10)
fig = px.bar(filtered_chb,
             x='billing_zip',
             y = 'chb_count',
             title = 'Top 10 CHB per Billing Zip',
             text = 'chb_count',
             labels = {'billing_zip' : 'Billing Zip', 'chb_count' : 'CHB Count'}
            )
fig.show()

billing_zip = the zip code filled in the billing details.
This plot shows the amount of CHB per billing zip, according to the plot we can assume that these zip codes might have a higher risk of CHB.

In [ ]:
# 2. Checking CHB over time in order to identify trends
chb_over_time = df.loc[df['order_status'] != 'approved'].groupby('order_date').size()
fig = px.line(chb_over_time,
              x= chb_over_time.index,
              y = chb_over_time,
              title = 'CHB Over Time',
              text = chb_over_time,
              labels = {'y' : 'CHB Count', 'order_date' : 'Order Date'},
             )
fig.update_traces(textposition='bottom center')

fig.update_xaxes(
    dtick="d1"
)

fig.add_trace(
    go.Scatter(x=chb_over_time.index,
               y=[chb_over_time.mean()] * len(chb_over_time),
               mode='lines',
               name='Avg CHB',
               line=dict(color='red')
              ),
    row=1, col=1
)

fig.update_layout(xaxis_tickangle=45)

fig.show()

The AVG amount of CHB is 11.6, when looking at the sum of CHB over time the data shows that it is fluctuating with 3 main extremes, therefore further investigation with other parameters is needed.

In [ ]:
# 3. Checking CHB per order source
chb_source = df.loc[df['order_status'] != 'approved'].groupby(['order_date','order_source']).size().reset_index().rename(columns={0:'CHB count'})
fig = px.line(chb_source,
              x= 'order_date',
              y = 'CHB count',
              title = 'CHB per Order Source',
              labels = {'y' : 'CHB Count', 'order_date' : 'Order Date', 'order_source': 'Order Source'},
              color = 'order_source',
              text = 'CHB count'
             )
fig.update_traces(textposition='bottom center')
fig.show()

In [ ]:
# Checking order source amounts
bar_df = df.groupby(by=['order_source','order_status']).order_id.count().reset_index().rename(columns={'order_id' :'count',
                                                                                                      'order_source' : 'Order Source',
                                                                                                      'order_status' : 'Order Status'})
fig = px.bar(bar_df,
             x='Order Source',
             y = 'count',
             color = 'Order Status',
             title = 'Order Source by Status breakdown',
             text = 'count',
             barmode='group',
             labels = {'count' : 'Count'}
            )
fig.update_traces(textposition='auto')
fig.show()

In [ ]:
percentage = 100 * df.groupby(by=['order_source', 'order_status'])['order_source'].count() / df.groupby(by=['order_source'])['order_source'].count()
percentage_df = percentage.reset_index(name='percentage')
percentage_df['percentage'] = percentage_df['percentage'].round(2)
percentage_df['text'] = percentage_df['percentage'].astype(str) + '%'
fig = px.bar(percentage_df,
             x='order_source',
             y='percentage',
             color='order_status',
             barmode='group',
             text='text',
             title = 'Order Source Percentage by Status breakdown',
             labels = {'order_status' : 'Order Status','order_source': 'Order Source'}
            )
fig.show()

When looking at the order status per source it seems like most CHB occur on the "Web", with that being said the rate of CHB is slightly higher on the "Mobile App". Further investigation is needed to understand what causes the differences in the CHB rate. When it comes to trends over time the data is not significant enough regarding "Mobile App" but it looks like the amount of CHB is fluctuating on the "Web" source.